In [1]:
import pandas as pd
import os
import statistics

In [2]:
results_directory = r"../result_files/2025-02-24 10-32-05/cpu"
files = [f for f in os.listdir(results_directory) if os.path.isfile(os.path.join(results_directory, f))]


In [23]:
info = {}

for f in files:
    if "_cpu" not in f:
        continue

    df = pd.read_csv(os.path.join(results_directory, f), sep=",")
    cpu_mean = round(df['cpu-percentage'].mean().item(), 3)


    # convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
    time_delta =  (df['timestamp'].max() - df['timestamp'].min()).total_seconds()

    f = f.replace("DistanceEstimationApproach.", "")
    f = f.replace("_cpu", "")
    f = f.replace(".csv", "")
    # remove leading run id
    f = f.split("_", 1)[1]
    if 'overhead' in f:
        f = f.replace("_overhead", "")

        if f not in info:
            info[f] = {
                "overhead_cpu_mean": [],
                "overhead_time_delta": [],
                "overhead_score": [],
                "cpu_mean": [],
                "time_delta": [],
                "score": []
            }

        info[f]["overhead_cpu_mean"] += [cpu_mean]
        info[f]["overhead_time_delta"] += [time_delta]
        # Score = cpu_mean * time (this is to account for the time when comparing cpu usage over time..)
        info[f]["overhead_score"] += [round(time_delta * cpu_mean, 3)]

    else:
        if f not in info:
            info[f] = {
                "overhead_cpu_mean": [],
                "overhead_time_delta": [],
                "overhead_score": [],
                "cpu_mean": [],
                "time_delta": [],
                "score": []
            }

        info[f]["cpu_mean"] += [cpu_mean]
        info[f]["time_delta"] += [time_delta]
        # Score = cpu_mean * time (this is to account for the time when comparing cpu usage over time..)
        info[f]["score"] += [round(time_delta * cpu_mean, 3)]

In [24]:
_df = pd.DataFrame(info)

for col in _df.columns:
    _df[col] = _df[col].apply(statistics.mean)
    _df[col] = round(_df[col], 3)

_df = _df.T
_df['cpu_cleaned'] = round(_df['cpu_mean'] - _df['overhead_cpu_mean'], 3)
_df['score_cleaned'] = round(_df['score'] - _df['overhead_score'], 3)
_df

,overhead_cpu_mean,overhead_time_delta,overhead_score,cpu_mean,time_delta,score,cpu_cleaned,score_cleaned
BRIDGE_NO_REC,34.073,198.100,6749.292,64.814,257.633,16697.660,30.741,9948.368
BRIDGE_SPLIT_NO_REC,36.260,205.700,7458.088,73.304,254.200,18633.603,37.044,11175.515
HAVERSINE,33.831,196.300,6640.845,33.376,198.300,6617.832,-0.455,-23.013
OSRM,10.581,479.633,5074.906,28.100,547.633,15388.644,17.519,10313.738
OVERHEAD_GRAPH_1024,33.653,198.100,6666.043,33.668,201.000,6767.079,0.015,101.036
OVERHEAD_GRAPH_256,33.707,197.767,6665.669,33.681,200.433,6750.387,-0.026,84.718
OVERHEAD_GRAPH_512,33.764,196.967,6650.199,33.601,200.800,6746.785,-0.163,96.586
WATER_GRAPH_CIRCUITY,34.253,200.967,6883.190,70.742,272.767,19295.753,36.489,12412.563
